In [32]:
def get_m_files(d="."):
    exts=["m"]
    for root, dirs, files in walk(d, topdown=False):
        for name in files:
            if os.path.splitext(name)[1].strip('.') in exts:
                yield os.path.abspath(os.path.join(root, name))

In [31]:
print(inspect.getsource(get_m_files))

def get_m_files(d="."):
    exts=["m"]
    for root, dirs, files in walk(d, topdown=False):
        for name in files:
            if os.path.splitext(name)[1].strip('.') in exts:
                yield os.path.abspath(os.path.join(root, name))



In [29]:
import inspect
print(inspect.getsource(convert_m_script_to_ipython_notebook))

def convert_m_script_to_ipython_notebook(m_file):
    nb = nbf.new_notebook()
    
    example, section = os.path.splitext(os.path.basename(m_file))[0].split("_")
    title = "{} - {}".format(re.sub(r"(\w)([A-Z])", r"\1 \2", example),
                             re.sub(r"(\w)([A-Z])", r"\1 \2", section))
    slug = "{}-{}".format(re.sub(r"(\w)([A-Z])", r"\1_\2", example.lower()),
                             re.sub(r"(\w)([A-Z])", r"\1_\2", section.lower()))
    with open (m_file, "r") as fid:
        lines=fid.readlines()
    
    last_mtplt=None
    for line in lines:
        mtplt = matlab_to_python_line_transmogrification(line)
        if last_mtplt is None:
            running_block=""           
            notebook_cells=list()
        else:
            if mtplt.cell_type != last_mtplt.cell_type:
                if last_mtplt.cell_type == 3:
                    cell_tmp = nbf.new_markdown_cell(running_block.strip())
                if last_mtplt.cell_type == 2:
                

In [21]:
def matlab_to_python_code_converter(code_block):
    assignments = re.compile("([\w_]+) ?= ? \[([^]]+)\]").findall(code_block)
    assignment_blocks=list()
    for assignment in assignments:
        variable=assignment[0]
        try:
            sin=Homunculus(assignment[1],assignment[0])
            assignment_blocks.append(sin)
        except:
            pass
    
for m in get_m_files():
    ipynb = os.path.splitext(m)[0]+"_06.ipynb"
    nb = convert_m_script_to_ipython_notebook(m)

Values = ['1/m']
Line   = 1/m
Values = ['0,1/m']
Line   = 0 1/m
Values = ['[A,[0,0,0,0']
Line   = [A,[0 0 0 0
Values = ['B', '[0,0']
Line   = B;[0 0


ValueError: too many values to unpack (expected 2)

In [16]:
# http://fma.wikia.com/wiki/Homunculus
class Homunculus(object):
    def __init__(self, matlab_line, variable):
        self.matlab_line=matlab_line.strip()
        self.python_line=matlab_line
        self.variable = variable
        self.alchemy()
        
    def alchemy(self):
        value = self.matlab_line
        value=value.replace("\t","")
        value=value.replace("\n","")
        for q in range(10): 
            value=value.replace("  "," ")
        values = [V.replace(" ",",") for V in value.split("; ")]
        if len(values)==1:
            values = values[0].split(";")
        
        pythonic = "numpy.array([["+("],[".join(values)+"]])")
        
        regex=r"""numpy.array\(\[\[([\d.]+):([\d.]+):([\d.]+)\]\]\)"""
        match = re.compile(regex).findall(pythonic)
        if len(match) == 1:
            start, step, end = match[0]
            pythonic = "numpy.arange({start},{end},{step})".format(start=start,end=end,step=step)
        regex=r"""numpy.array\(\[\[([\d.]+):([\d.]+)\]\]\)"""
        match = re.compile(regex).findall(pythonic)
        if len(match) == 1:
            start, end = match[0]
            pythonic = "numpy.arange({start},{end})".format(start=start,end=end,step=step)
        try:
            eval(pythonic)
        except NameError:
            pass
        except:
            print("Values = {}".format(values))
            print("Line   = {}".format(self.matlab_line))
            if False:
                raise
            else:
                pythonic = "# "+pythonic
        self.python_line = pythonic
        
    def __repr__(self):
        return self.variable+" = "+self.python_line

In [30]:
def convert_m_script_to_ipython_notebook(m_file):
    nb = nbf.new_notebook()
    
    example, section = os.path.splitext(os.path.basename(m_file))[0].split("_")
    title = "{} - {}".format(re.sub(r"(\w)([A-Z])", r"\1 \2", example),
                             re.sub(r"(\w)([A-Z])", r"\1 \2", section))
    slug = "{}-{}".format(re.sub(r"(\w)([A-Z])", r"\1_\2", example.lower()),
                             re.sub(r"(\w)([A-Z])", r"\1_\2", section.lower()))
    with open (m_file, "r") as fid:
        lines=fid.readlines()
    
    last_mtplt=None
    for line in lines:
        mtplt = matlab_to_python_line_transmogrification(line)
        if last_mtplt is None:
            running_block=""           
            notebook_cells=list()
        else:
            if mtplt.cell_type != last_mtplt.cell_type:
                if last_mtplt.cell_type == 3:
                    cell_tmp = nbf.new_markdown_cell(running_block.strip())
                if last_mtplt.cell_type == 2:
                    cell_tmp = nbf.new_code_cell(running_block.strip())
                notebook_cells.append(cell_tmp)
                running_block=str_tmp=str(mtplt)
                last_mtplt = mtplt
                continue
        str_tmp=str(mtplt)
        if len(str_tmp) == 0:
            continue
        if running_block.endswith("\n"):
            running_block=(running_block+str_tmp)
        else:
            running_block=(running_block+" "+str_tmp)
        last_mtplt = mtplt
        

        
    ## Start Cells
    nb["cells"].append(nbf.new_markdown_cell(r"""<div id="toc"></div>"""))
        
    ## Body cells
    for notebook_cell in notebook_cells:
        cleaned_cell=cell_cleaner_upperer(notebook_cell)
        nb["cells"].append(cleaned_cell)
        
    ## End Cells
    nb["cells"].append(nbf.new_code_cell(r"""%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')"""))

    nikola_metadata=dict()
    nikola_metadatas=dict()
    nikola_metadata["author"]=""
    nikola_metadata["category"]=example
    nikola_metadata["date"]=""
    nikola_metadata["description"]=""
    nikola_metadata["link"]=""
    nikola_metadata["slug"]=slug
    nikola_metadata["tags"]=section
    nikola_metadata["title"]=title
    nikola_metadata["type"]=""
    nikola_metadatas["nikola"]=nikola_metadata
    nb["metadata"]=nikola_metadatas
    return nb

In [6]:
def cell_cleaner_upperer(notebook_cell):
    source = notebook_cell["source"]
    if notebook_cell["cell_type"] is "markdown":
        source=markdown_cleaner_upperer(source)
    if notebook_cell["cell_type"] is "code":
        source=matlab_to_python_code_converter(source)
    #print(source, end="")
    notebook_cell["source"] = source
    return notebook_cell
def markdown_cleaner_upperer(markdown_block):
     # Non inline latex
    markdown_block=re.sub(r"\$\$([^$]+)\$\$",r"\n\n$$\n\1\n$$\n\n",markdown_block)
    # Embedded Functions
    markdown_block=re.sub(r"\|([\w_]+)\|",r"```\1```",markdown_block)
    # Links to other
    markdown_block=re.sub(r"< \?example=([\w]+)&section=([\w]+) ([^>]+)>",
                  r"[\3](/examples/\1/\1-\2.ipynb)",markdown_block)
    return markdown_block

import numpy
import re

Nope, this is taking too long. Comment out bad code.